### Importing Necessary Libraries


In [11]:
import numpy as np
import pandas as pd

### Extraction Layer

In [25]:
sales_df = pd.read_csv(r'C:\Users\nonso\Desktop\new sql assignment\EngrNony-New-SQL-Assignment\dataset\raw_data\Sales Dataset.csv')

In [26]:
sales_df.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,Order Date,CustomerName,State,City,Year-Month
0,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2023-06-27,David Padilla,Florida,Miami,2023-06
1,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2024-12-27,Connor Morgan,Illinois,Chicago,2024-12
2,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2021-07-25,Robert Stone,New York,Buffalo,2021-07
3,B-26776,4975,1330,14,Electronics,Printers,UPI,2023-06-27,David Padilla,Florida,Miami,2023-06
4,B-26776,4975,1330,14,Electronics,Printers,UPI,2024-12-27,Connor Morgan,Illinois,Chicago,2024-12


In [235]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1194 entries, 0 to 1193
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Order ID      1194 non-null   object        
 1   Amount        1194 non-null   int64         
 2   Profit        1194 non-null   int64         
 3   Quantity      1194 non-null   int64         
 4   Category      1194 non-null   object        
 5   Sub-Category  1194 non-null   object        
 6   PaymentMode   1194 non-null   object        
 7   Order Date    1194 non-null   datetime64[ns]
 8   CustomerName  1194 non-null   object        
 9   State         1194 non-null   object        
 10  City          1194 non-null   object        
 11  Year-Month    1194 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(7)
memory usage: 112.1+ KB


In [236]:
sales_df.columns

Index(['Order ID', 'Amount', 'Profit', 'Quantity', 'Category', 'Sub-Category',
       'PaymentMode', 'Order Date', 'CustomerName', 'State', 'City',
       'Year-Month'],
      dtype='object')

### Data Cleaning Step

In [237]:
#convert order date from string to datetime object
sales_df['Order Date'] = pd.to_datetime(sales_df['Order Date'])
sales_df['Year-Month'] = pd.to_datetime(sales_df['Year-Month'])

In [238]:
sales_df.columns

Index(['Order ID', 'Amount', 'Profit', 'Quantity', 'Category', 'Sub-Category',
       'PaymentMode', 'Order Date', 'CustomerName', 'State', 'City',
       'Year-Month'],
      dtype='object')

In [247]:
# Sales Table
sales_table = sales_df[['Order ID', 'Amount', 'Profit', 'Quantity', 'Category', 'Sub-Category', 'PaymentMode', 'Order Date', 'CustomerName', 'State', 'City', 'Year-Month']].copy().reset_index(drop=True)
sales_table.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,Order Date,CustomerName,State,City,Year-Month
0,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2023-06-27,David Padilla,Florida,Miami,2023-06-01
1,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2024-12-27,Connor Morgan,Illinois,Chicago,2024-12-01
2,B-26776,9726,1275,5,Electronics,Electronic Games,UPI,2021-07-25,Robert Stone,New York,Buffalo,2021-07-01
3,B-26776,4975,1330,14,Electronics,Printers,UPI,2023-06-27,David Padilla,Florida,Miami,2023-06-01
4,B-26776,4975,1330,14,Electronics,Printers,UPI,2024-12-27,Connor Morgan,Illinois,Chicago,2024-12-01


In [248]:
# Save table to csv
sales_table.to_csv(r'C:\Users\nonso\Desktop\new sql assignment\EngrNony-New-SQL-Assignment\dataset\table_data/sales.csv', index=False)

### Data Loading

In [249]:
!pip install psycopg2

In [250]:
import psycopg2


In [251]:
def db_connection():
    connection = psycopg2.connect(
        host = 'localhost',
        database = 'Sales_performance',
        user = 'postgres',
        password = 'password'
    )
    return connection

In [252]:
conn = db_connection()

In [257]:
# Create our SQL Table
def create_table():
    conn = db_connection()
    cursor = conn.cursor()
    create_table_query = '''
                            CREATE SCHEMA IF NOT EXISTS analysis;

                            DROP TABLE IF EXISTS analysis.sales CASCADE;


                            CREATE TABLE IF NOT EXISTS analysis.sales (
                                "Order ID" VARCHAR,
                                Amount FLOAT,
                                Profit FLOAT,
                                Quantity INTEGER,
                                Category TEXT,
                                "Sub-Category" TEXT,
                                PaymentMode TEXT,
                                "Order Date" DATE,
                                CustomerName TEXT,
                                State TEXT,
                                City TEXT,
                                "Year-Month" DATE
                            );
                        '''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

In [258]:
create_table()

In [262]:
import csv
def load_from_csv(csv_path):
    conn = db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute('''
                            INSERT into analysis.sales("Order ID", Amount, Profit, Quantity, Category, "Sub-Category", PaymentMode, "Order Date", CustomerName, State, city, "Year-Month")
                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);''',
                            row
                        )
    conn.commit()
    cursor.close()
    conn.close()

csv_file_path = r'C:\Users\nonso\Desktop\new sql assignment\EngrNony-New-SQL-Assignment\dataset\table_data\sales.csv'
load_from_csv(csv_file_path)

